# NanoAOD provenance checker
Makes sure all files are from the same gridpack

In [11]:
import subprocess
from concurrent.futures import ThreadPoolExecutor,as_completed
import random

In [13]:
def getGridpack(f):
    ret = subprocess.run("edmProvDump -ef externalLHEProducer " + f + " | grep /cvmfs/cms.cern.ch/phys_generator/", shell=True, check=True, stdout=subprocess.PIPE)
    return ret.stdout.decode()
eos_server = "root://eos.grif.fr"
eos_commands = ["eos", "-b", eos_server]
eos_basepath_crab = "/eos/grif/cms/llr/store/user/tcuisset/boostedTaus/2018_prod/"
eos_output_path = "/eos/grif/cms/llr/store/user/tcuisset/boostedTaus/2018_MC/"
def run_eos_cmd(*args):
    ret = subprocess.run([ *eos_commands, *args], stdout=subprocess.PIPE, check=True)
    return ret.stdout.decode().splitlines()

In [10]:
getGridpack("root://eos.grif.fr///eos/grif/cms/llr/store/user/tcuisset/boostedTaus/2018_MC/ST_t-channel_top_4f_InclusiveDecays/NANO_NANO_10.root")

b"  args: vstring tracked  = {'/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/madgraph/V5_2.6.5/st_tch_4f_ckm_UL_powheg_madspin/st_tch_4f_ckm_top_13TeV_UL_powheg_madspin_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n"

In [5]:
subprocess.run("voms-proxy-info", shell=True)

subject   : /DC=ch/DC=cern/OU=Organic Units/OU=Users/CN=tcuisset/CN=858886/CN=Theo Cuisset/CN=436848596
issuer    : /DC=ch/DC=cern/OU=Organic Units/OU=Users/CN=tcuisset/CN=858886/CN=Theo Cuisset
identity  : /DC=ch/DC=cern/OU=Organic Units/OU=Users/CN=tcuisset/CN=858886/CN=Theo Cuisset
type      : RFC3820 compliant impersonation proxy
strength  : 2048
path      : /grid_mnt/data__data.polcms/cms/cuisset/ZHbbtautau/framework/nanoaod_base_analysis/x509up
timeleft  : 191:59:56
key usage : Digital Signature, Key Encipherment


CompletedProcess(args='voms-proxy-info', returncode=0)

In [28]:
sample=50
with ThreadPoolExecutor(max_workers=30) as exc:
    futs = {}
    for dataset in run_eos_cmd("ls", eos_output_path):
        files = run_eos_cmd("ls", eos_output_path + "/" + dataset)
        futs[dataset] = [exc.submit(getGridpack, "root://eos.grif.fr///eos/grif/cms/llr/store/user/tcuisset/boostedTaus/2018_MC/" + dataset + "/" + file) for file in (random.sample(files, sample) if len(files)>sample else files)]


In [20]:
futs["DYJetsToLL_0J-amcatnloFXFX"][0].result()

"  args: vstring tracked  = {'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc7_amd64_gcc700/13TeV/madgraph/V5_2.6.5/dy_jetbinned_fxfx/dyellell0j_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n"

In [29]:
grids = {dataset : set(fut.result() for fut in futs_dataset) for dataset, futs_dataset in futs.items()}
grids

{'DYJetsToLL_0J-amcatnloFXFX': {"  args: vstring tracked  = {'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc7_amd64_gcc700/13TeV/madgraph/V5_2.6.5/dy_jetbinned_fxfx/dyellell0j_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n"},
 'DYJetsToLL_1J-amcatnloFXFX': {"  args: vstring tracked  = {'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc7_amd64_gcc700/13TeV/madgraph/V5_2.6.5/dy_jetbinned_fxfx/dyellell1j_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n"},
 'DYJetsToLL_2J-amcatnloFXFX': {"  args: vstring tracked  = {'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc7_amd64_gcc700/13TeV/madgraph/V5_2.6.5/dy_jetbinned_fxfx/dyellell2j_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n"},
 'DYJetsToLL_LHEFilterPtZ-0To50-amcatnloFXFX': {"  args: vstring tracked  = {'/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/madgraph/V5_2.6.5/EWparamSync/DYJetsToLL_012j_Ptbin_5f_NLO_FXFX_updated/DYJetsToLL_012j_Zpt-0To80_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_1

In [30]:
max((len(g) for g in grids.values()))

1

In [31]:
grids_nice = {dataset : next(iter(s))[len('{"  args: vstring tracked  = {'):] for dataset, s in grids.items()}
grids_nice

{'DYJetsToLL_0J-amcatnloFXFX': "cvmfs/cms.cern.ch/phys_generator/gridpacks/slc7_amd64_gcc700/13TeV/madgraph/V5_2.6.5/dy_jetbinned_fxfx/dyellell0j_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n",
 'DYJetsToLL_1J-amcatnloFXFX': "cvmfs/cms.cern.ch/phys_generator/gridpacks/slc7_amd64_gcc700/13TeV/madgraph/V5_2.6.5/dy_jetbinned_fxfx/dyellell1j_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n",
 'DYJetsToLL_2J-amcatnloFXFX': "cvmfs/cms.cern.ch/phys_generator/gridpacks/slc7_amd64_gcc700/13TeV/madgraph/V5_2.6.5/dy_jetbinned_fxfx/dyellell2j_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n",
 'DYJetsToLL_LHEFilterPtZ-0To50-amcatnloFXFX': "cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/madgraph/V5_2.6.5/EWparamSync/DYJetsToLL_012j_Ptbin_5f_NLO_FXFX_updated/DYJetsToLL_012j_Zpt-0To80_5f_NLO_FXFX_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz'}\n",
 'DYJetsToLL_LHEFilterPtZ-100To250-amcatnloFXFX': "cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/m